### Step 1. Import required Python packages
Required packages (installation instructions provided in README.md):
 * Generic: numpy
 * Geometry manipulation: shapely, geopandas
 * Shortest path: sp (https://github.com/cb-cities/sp)
 * Plotting: matplotlib

In [17]:
### plotting
import pandas as pd 
import matplotlib.pyplot as plt

### transit simulation
from model.transit_sim_model import Network, Trains, Travelers

### fix random seed
import numpy as np
np.random.seed(0)

### Step 2. Process GTFS schedules
Need the following tables:
 * stop_times.txt: schedule info
 * trips.txt: map trip_id to route_id
 * stops.txt: get stop coordinates (for visualization)

In [18]:
### input and output path
in_path = ### replace with path to input folder
out_path = ### replace with path to output folder
start_time = 5*3600 ### simulation start time in seconds since 12 AM
end_time = 11*3600 ### simulation end time in seconds  since 12 AM 
time_step = 20 ### simulation time step size in seconds

### read in GTFS files
stop_times_file = ### path to GTFS stop_times.txt file
trips_file = ### path to GTFS trips.txt file
stops_file = ### path to GTFS stops.txt file

### only keep results with this service id
service_id = ### GTFS service_id, can be found in calendar.txt file
scen_nm = 'test'

### create all trains from GTFS
all_trains = Trains()
all_nodes, all_links = all_trains.schedule_and_network_from_gtfs(
    stop_times_file, trips_file, stops_file, service_id)

### create network from nodes and links
network = Network(all_nodes, all_links)

In [ ]:
### display and export schedule
all_trains.schedule_df.to_csv('{}/{}_schedule.csv'.format(out_path, scen_nm), index=False)
all_trains.schedule_df.head(1)

### display and export network
display(network.all_nodes.head(1))
display(network.all_links.head(1))
network.all_links.to_csv('{}/{}_links.csv'.format(out_path, scen_nm), index=False)
network.all_nodes.to_csv('{}/{}_nodes.csv'.format(out_path, scen_nm), index=False)

### Step 3. Travel demand
 * random demand
 * or, input csv with columns *traveler_id*, *origin_nid*, *destin_nid*, *departure_time*

In [19]:
def set_demand(scen_nm, network):
    travelers_df = pd.read_csv('{}/{}_od.csv'.format(in_path, scen_nm))

    station_to_node_dict = {getattr(row, 'stop_id'): getattr(row, 'node_id') for row in network.all_nodes.itertuples()}
    travelers_df['origin_nid'] = travelers_df['enter_station'].map(station_to_node_dict)#.astype(int)
    travelers_df['destin_nid'] = travelers_df['exit_station'].map(station_to_node_dict)#.astype(int)
    travelers_df['traveler_id'] = np.arange(travelers_df.shape[0])
    travelers_df = travelers_df[travelers_df['origin_nid'] != travelers_df['destin_nid']]

    travelers = Travelers()
    travelers.travelers_df = travelers_df[['traveler_id', 'origin_nid', 'destin_nid', 'departure_time']].copy()#.iloc[1871:1875]
    travelers.find_routes(network.network_g, network.station_id_nm_dict, network.station_id_route_dict)
    travelers.set_initial_status(network.station_id_nm_dict)

    print(travelers.travelers_df.shape)
    display(travelers.travelers_df.tail())
    
    return travelers

### Step 4. Run the simulation

In [20]:
def save_agg_results(network, trains, travelers, t, scen_nm=''):
    ### save train results
    train_positions = trains.get_all_train_positions(network)
    train_positions.to_csv('{}/train_outputs/train_outputs_{}_{}.csv'.format(out_path, scen_nm, t), index=False)
    ### save aggregated traveler results
    traveler_locations = travelers.travelers_df[['traveler_status', 'association']].fillna(np.nan).groupby(
            ['traveler_status', 'association']).size().to_frame(
            name='num_travelers').reset_index(drop=False)
    traveler_locations.to_csv('{}/traveler_outputs/agg_traveler_outputs_{}_{}.csv'.format(out_path, scen_nm, t), 
                              index=False)

def save_indiv_results(travelers, t, scen_nm=''):
    ### save individual traveler results
    travelers.travelers_df.to_csv('{}/traveler_outputs/indiv_traveler_outputs_{}_{}.csv'.format(out_path, scen_nm, t), 
                                  index=False)
    
def save_new_board(new_board, t, scen_nm = ''):
    #### list of newly boarded travelers
    if new_board.shape[0]>0:
        new_board.to_csv('{}/traveler_outputs/boarding_traveler_outputs_{}_{}.csv'.format(out_path, scen_nm, t), 
                                  index=False)
        
def run_simulation(network, all_trains, travelers):
    
    t_init, t_end, t_step = start_time, end_time, time_step
    trace_ods = []
    trace_list = []
    scen_spec = 'spec' ### some simulation specific tags

    agent_status = None
    for t in range(t_init, t_end, t_step):

        ### update train location
        all_trains.update_location_occupancy(t)

        ### update traveler status
        new_board = travelers.traveler_update(network, all_trains, t)

        ### print and plot results
        save_agg_results(network, all_trains, travelers, t, scen_nm=scen_nm+'_'+scen_spec)
        ### save boarding data for analysis
        # save_new_board(new_board, t, scen_nm=scen_nm)

        trace_list.append(travelers.travelers_df.loc[travelers.travelers_df['traveler_id'].isin(trace_ods), 
                                          ['traveler_id', 'update_time', 'traveler_status', 'association']])

        if (t-t_init)%300==0:
            # save_indiv_results(travelers, t, scen_nm=scen_nm+'_'+scen_spec)
            print('Simulation at {}:{:02}am, {}'.format(t//3600, (t%3600)//60, t))

    ### agent trace
    trace_df = pd.concat(trace_list)
    trace_df = trace_df.groupby(['traveler_id', 'traveler_status', 'association']).first().reset_index()
    trace_df.to_csv('{}/traveler_outputs/traveler_trace_{}.csv'.format(
        out_path, scen_nm+'_'+scen_spec), index=False)
    
    ### final traveler status
    travelers.travelers_df.to_csv('{}/traveler_outputs/final_traveler_status_{}.csv'.format(
        out_path, scen_nm+'_'+scen_spec), index=False)

In [ ]:
### reset all traveler status to pre-departure
travelers = set_demand(scen_nm, network)

### run simulation and save outputs
run_simulation(network, all_trains, travelers)